# 🌡️ Descarga y Procesamiento de Datos de Temperatura

Este notebook descarga datos de temperatura desde InfluxDB, los procesa y genera un archivo final limpio.

## 📋 Proceso:
1. **Configuración** inicial y conexión a InfluxDB
2. **Descarga** de datos históricos (TempModFixed) y recientes (fixed_plant_atamo_1)
3. **Corrección** del formato deformado (sensores en filas separadas)
4. **Combinación** de ambos datasets
5. **Limpieza** final y generación de `data_temp.csv`


In [13]:
# ============================================================================
# 📚 IMPORTACIONES Y CONFIGURACIÓN INICIAL
# ============================================================================

import pandas as pd
import numpy as np
import os
import sys
import logging
import gc
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Configuración de InfluxDB
INFLUX_CONFIG = {
    'url': "http://146.83.153.212:27017",
    'token': "piDbFR_bfRWO5Epu1IS96WbkNpSZZCYgwZZR29PcwUsxXwKdIyLMhVAhU4-5ohWeXIsX7Dp_X-WiPIDx0beafg==",
    'org': "atamostec",
    'timeout': 300000
}

# Configuración de fechas y rutas
START_DATE = pd.to_datetime('01/07/2024', dayfirst=True).tz_localize('UTC')
END_DATE = pd.to_datetime('30/06/2025', dayfirst=True).tz_localize('UTC')
OUTPUT_DIR = "/home/nicole/SR/SOILING/datos"
CUTOFF_DATE = pd.to_datetime('2024-12-05 16:00:00+00:00')  # Fecha donde cambia la fuente

os.makedirs(OUTPUT_DIR, exist_ok=True)

logger.info(f"🌡️ DESCARGA DE DATOS DE TEMPERATURA")
logger.info(f"📅 Período: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio: {OUTPUT_DIR}")
print(f"🐍 Python: {sys.version}")


2025-06-25 09:22:36 - INFO - 🌡️ DESCARGA DE DATOS DE TEMPERATURA
2025-06-25 09:22:36 - INFO - 📅 Período: 2024-07-01 a 2025-06-30
2025-06-25 09:22:36 - INFO - 📁 Directorio: /home/nicole/SR/SOILING/datos


🐍 Python: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [14]:
# ============================================================================
# 🔌 CLASE PARA MANEJO DE INFLUXDB
# ============================================================================

class InfluxDBManager:
    def __init__(self, config):
        self.config = config
        self.client = None
        self.query_api = None
        
    def connect(self):
        try:
            self.client = InfluxDBClient(
                url=self.config['url'],
                token=self.config['token'],
                org=self.config['org'],
                timeout=self.config['timeout']
            )
            self.query_api = self.client.query_api()
            logger.info("✅ Cliente InfluxDB inicializado")
            return True
        except Exception as e:
            logger.error(f"❌ Error conectando a InfluxDB: {e}")
            return False
    
    def query_influxdb(self, bucket, tables, attributes, start_date, end_date):
        try:
            # Construir filtros correctamente para Flux
            if len(tables) == 1:
                tables_filter = f'r._measurement == "{tables[0]}"'
            else:
                tables_conditions = [f'r._measurement == "{table}"' for table in tables]
                tables_filter = ' or '.join(tables_conditions)
            
            if len(attributes) == 1:
                fields_filter = f'r._field == "{attributes[0]}"'
            else:
                fields_conditions = [f'r._field == "{field}"' for field in attributes]
                fields_filter = ' or '.join(fields_conditions)
            
            query = f"""
            from(bucket: "{bucket}")
            |> range(start: {start_date.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_date.strftime('%Y-%m-%dT%H:%M:%SZ')})
            |> filter(fn: (r) => {tables_filter})
            |> filter(fn: (r) => {fields_filter})
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
            """
            
            logger.info(f"📊 Consultando: bucket={bucket}, tables={tables}")
            logger.info(f"📅 Período: {start_date.strftime('%Y-%m-%dT%H:%M:%SZ')} a {end_date.strftime('%Y-%m-%dT%H:%M:%SZ')}")
            
            result = self.query_api.query_data_frame(query)
            
            if result is not None and not result.empty:
                logger.info(f"✅ Datos obtenidos: {len(result)} registros")
                return result
            else:
                logger.warning("⚠️ No se encontraron datos para el período consultado")
                return None
                
        except Exception as e:
            logger.error(f"❌ Error en consulta: {e}")
            return None
    
    def disconnect(self):
        if self.client:
            self.client.close()
            logger.info("🔌 Conexión a InfluxDB cerrada")

logger.info("✅ Clase InfluxDBManager definida")


2025-06-25 09:22:37 - INFO - ✅ Clase InfluxDBManager definida


In [15]:
# ============================================================================
# 🔧 FUNCIÓN PARA CORREGIR FORMATO DEFORMADO
# ============================================================================

def fix_temperature_format(df):
    """
    Corrige el formato deformado donde cada sensor está en filas separadas.
    Consolida los datos para que todos los sensores estén en una sola fila por timestamp.
    """
    logger.info("🔧 Aplicando corrección de formato...")
    
    try:
        temp_cols = ['1TE416(C)', '1TE417(C)', '1TE418(C)', '1TE419(C)']
        
        # Contar valores no nulos por fila
        df['non_null_count'] = df[temp_cols].notna().sum(axis=1)
        
        # Separar datos correctos (4 valores) de deformados (1 valor)
        df_correct = df[df['non_null_count'] == 4].copy()
        df_deformed = df[df['non_null_count'] == 1].copy()
        
        logger.info(f"📊 Datos correctos: {len(df_correct):,} filas")
        logger.info(f"🔧 Datos deformados: {len(df_deformed):,} filas")
        
        if len(df_deformed) == 0:
            logger.info("✅ No hay datos deformados que corregir")
            return df_correct.drop('non_null_count', axis=1)
        
        # Procesar datos deformados
        logger.info("🔄 Consolidando datos deformados...")
        
        # Redondear timestamps para agrupar
        df_deformed['_time_rounded'] = df_deformed.index.round('1min')
        
        consolidated_rows = []
        
        for timestamp, group in df_deformed.groupby('_time_rounded'):
            # Crear fila consolidada
            consolidated_row = {
                '_time': timestamp,
                '1TE416(C)': np.nan,
                '1TE417(C)': np.nan,
                '1TE418(C)': np.nan,
                '1TE419(C)': np.nan
            }
            
            # Extraer valores de cada sensor
            for _, row in group.iterrows():
                for col in temp_cols:
                    if pd.notna(row[col]):
                        consolidated_row[col] = row[col]
            
            # Solo agregar si tiene al menos un valor
            if any(pd.notna(consolidated_row[col]) for col in temp_cols):
                consolidated_rows.append(consolidated_row)
        
        if consolidated_rows:
            df_consolidated = pd.DataFrame(consolidated_rows)
            df_consolidated.set_index('_time', inplace=True)
            logger.info(f"✅ Datos consolidados: {len(df_consolidated):,} filas")
            
            # Combinar datos correctos y consolidados
            df_final = pd.concat([df_correct.drop('non_null_count', axis=1), df_consolidated])
            df_final = df_final.sort_index()
            
            logger.info(f"📊 Total final: {len(df_final):,} filas")
            return df_final
        else:
            logger.warning("⚠️ No se pudieron consolidar los datos deformados")
            return df_correct.drop('non_null_count', axis=1)
            
    except Exception as e:
        logger.error(f"❌ Error en corrección de formato: {e}")
        return df.drop('non_null_count', axis=1, errors='ignore')

logger.info("✅ Función de corrección de formato definida")


2025-06-25 09:22:38 - INFO - ✅ Función de corrección de formato definida


In [16]:
# ============================================================================
# 📥 FUNCIÓN PRINCIPAL DE DESCARGA
# ============================================================================

def download_temperature_data(influx_client, start_date, end_date, output_dir):
    """
    Descarga datos de temperatura de ambas fuentes y los procesa.
    """
    logger.info("🌡️ Iniciando descarga de datos de temperatura...")
    
    try:
        bucket = "PSDA"
        attributes = ["1TE416(C)", "1TE417(C)", "1TE418(C)", "1TE419(C)"]
        
        all_dataframes = []
        
        # PERÍODO 1: Datos históricos (TempModFixed)
        logger.info(f"\n🔄 DESCARGANDO DATOS HISTÓRICOS (TempModFixed)...")
        logger.info(f"📅 Período: {start_date.strftime('%Y-%m-%d')} a {CUTOFF_DATE.strftime('%Y-%m-%d')}")
        
        current_date = start_date
        period_1_end = min(CUTOFF_DATE, end_date)
        
        while current_date < period_1_end:
            week_end = min(current_date + timedelta(days=7), period_1_end)
            
            df_temp = influx_client.query_influxdb(bucket, ["TempModFixed"], attributes, current_date, week_end)
            
            if df_temp is not None and not df_temp.empty:
                # Procesar DataFrame
                if '_time' in df_temp.columns:
                    df_temp.set_index('_time', inplace=True)
                
                if not isinstance(df_temp.index, pd.DatetimeIndex):
                    df_temp.index = pd.to_datetime(df_temp.index)
                
                # Filtrar horario (13:00-18:00)
                df_temp = df_temp.between_time('13:00', '18:00')
                
                if not df_temp.empty:
                    # Limpiar columnas
                    cols_to_drop = [col for col in ['result', 'table', '_start', '_stop', '_measurement'] if col in df_temp.columns]
                    if cols_to_drop:
                        df_temp.drop(columns=cols_to_drop, inplace=True)
                    
                    all_dataframes.append(df_temp)
                    logger.info(f"   ✅ Semana procesada: {len(df_temp):,} registros")
            
            current_date = week_end
            gc.collect()
        
        # PERÍODO 2: Datos recientes (fixed_plant_atamo_1)
        if CUTOFF_DATE < end_date:
            logger.info(f"\n🔄 DESCARGANDO DATOS RECIENTES (fixed_plant_atamo_1)...")
            logger.info(f"📅 Período: {CUTOFF_DATE.strftime('%Y-%m-%d')} a {end_date.strftime('%Y-%m-%d')}")
            
            current_date = CUTOFF_DATE
            while current_date < end_date:
                week_end = min(current_date + timedelta(days=7), end_date)
                
                df_temp = influx_client.query_influxdb(bucket, ["fixed_plant_atamo_1"], attributes, current_date, week_end)
                
                if df_temp is not None and not df_temp.empty:
                    # Procesar DataFrame
                    if '_time' in df_temp.columns:
                        df_temp.set_index('_time', inplace=True)
                    
                    if not isinstance(df_temp.index, pd.DatetimeIndex):
                        df_temp.index = pd.to_datetime(df_temp.index)
                    
                    # Filtrar horario (13:00-18:00)
                    df_temp = df_temp.between_time('13:00', '18:00')
                    
                    if not df_temp.empty:
                        # Limpiar columnas
                        cols_to_drop = [col for col in ['result', 'table', '_start', '_stop', '_measurement'] if col in df_temp.columns]
                        if cols_to_drop:
                            df_temp.drop(columns=cols_to_drop, inplace=True)
                        
                        all_dataframes.append(df_temp)
                        logger.info(f"   ✅ Semana procesada: {len(df_temp):,} registros")
                
                current_date = week_end
                gc.collect()
        
        if not all_dataframes:
            logger.error("❌ No se obtuvieron datos")
            return False
        
        # COMBINAR TODOS LOS DATOS
        logger.info(f"\n📊 COMBINANDO Y PROCESANDO TODOS LOS DATOS...")
        df_combined = pd.concat(all_dataframes, ignore_index=False)
        del all_dataframes
        gc.collect()
        
        # Procesar datos combinados
        df_combined = df_combined.sort_index()
        df_combined = df_combined[~df_combined.index.duplicated(keep='first')]
        
        logger.info(f"📊 Datos antes de corrección: {len(df_combined):,} registros")
        
        # APLICAR CORRECCIÓN DE FORMATO
        df_corrected = fix_temperature_format(df_combined)
        
        # GUARDAR ARCHIVO FINAL
        final_file = os.path.join(output_dir, 'data_temp.csv')
        logger.info(f"💾 Guardando archivo final: {final_file}")
        df_corrected.to_csv(final_file)
        
        # ESTADÍSTICAS FINALES
        file_size_mb = os.path.getsize(final_file) / (1024*1024)
        
        logger.info(f"\n" + "="*70)
        logger.info(f"✅ DESCARGA COMPLETADA EXITOSAMENTE")
        logger.info(f"📊 Registros finales: {len(df_corrected):,}")
        logger.info(f"📅 Rango: {df_corrected.index.min()} a {df_corrected.index.max()}")
        logger.info(f"📁 Archivo: data_temp.csv ({file_size_mb:.2f} MB)")
        
        # Estadísticas por sensor
        for col in df_corrected.columns:
            valid_count = df_corrected[col].notna().sum()
            percentage = valid_count/len(df_corrected)*100
            logger.info(f"🌡️ {col}: {valid_count:,} valores ({percentage:.1f}%)")
        
        logger.info("="*70)
        
        return True
        
    except Exception as e:
        logger.error(f"❌ Error en la descarga: {e}")
        import traceback
        logger.error(f"🔍 Detalles: {traceback.format_exc()}")
        return False

logger.info("✅ Función principal de descarga definida")


2025-06-25 09:22:39 - INFO - ✅ Función principal de descarga definida


In [17]:
# ============================================================================
# 🚀 EJECUTAR DESCARGA COMPLETA
# ============================================================================

logger.info("\n🚀 INICIANDO DESCARGA COMPLETA DE DATOS DE TEMPERATURA")
logger.info("="*80)

# Inicializar cliente InfluxDB
influx_manager = InfluxDBManager(INFLUX_CONFIG)

try:
    if not influx_manager.connect():
        logger.error("❌ No se pudo establecer conexión con InfluxDB")
    else:
        # Ejecutar descarga completa
        success = download_temperature_data(influx_manager, START_DATE, END_DATE, OUTPUT_DIR)
        
        if success:
            logger.info("\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!")
            logger.info("📋 El archivo data_temp.csv está disponible y listo para usar")
            logger.info("🧹 Formato corregido automáticamente")
        else:
            logger.error("❌ ERROR EN EL PROCESO DE DESCARGA")

except Exception as e:
    logger.error(f"❌ Error general: {e}")
    import traceback
    logger.error(f"🔍 Detalles del error: {traceback.format_exc()}")
    
finally:
    influx_manager.disconnect()

logger.info("\n🏁 PROCESO FINALIZADO")


2025-06-25 09:22:40 - INFO - 
🚀 INICIANDO DESCARGA COMPLETA DE DATOS DE TEMPERATURA
2025-06-25 09:22:40 - INFO - ================================================================================
2025-06-25 09:22:41 - INFO - ✅ Cliente InfluxDB inicializado
2025-06-25 09:22:41 - INFO - 🌡️ Iniciando descarga de datos de temperatura...
2025-06-25 09:22:41 - INFO - 
🔄 DESCARGANDO DATOS HISTÓRICOS (TempModFixed)...
2025-06-25 09:22:41 - INFO - 📅 Período: 2024-07-01 a 2024-12-05
2025-06-25 09:22:41 - INFO - 📊 Consultando: bucket=PSDA, tables=['TempModFixed']
2025-06-25 09:22:41 - INFO - 📅 Período: 2024-07-01T00:00:00Z a 2024-07-08T00:00:00Z
2025-06-25 09:22:48 - INFO - ✅ Datos obtenidos: 120934 registros
2025-06-25 09:22:48 - INFO -    ✅ Semana procesada: 25,194 registros
2025-06-25 09:22:48 - INFO - 📊 Consultando: bucket=PSDA, tables=['TempModFixed']
2025-06-25 09:22:48 - INFO - 📅 Período: 2024-07-08T00:00:00Z a 2024-07-15T00:00:00Z
2025-06-25 09:22:54 - INFO - ✅ Datos obtenidos: 120864 regis

In [18]:
# ============================================================================
# 🧹 LIMPIEZA DE ARCHIVOS TEMPORALES (OPCIONAL)
# ============================================================================

def cleanup_temp_files():
    """
    Elimina archivos temporales que ya no son necesarios.
    Solo ejecutar si se confirma que data_temp.csv está correcto.
    """
    logger.info("🧹 Iniciando limpieza de archivos temporales...")
    
    temp_files_to_remove = [
        'temp_historical_TempModFixed.csv',
        'temp_recent_fixed_plant_atamo_1.csv',
        'temp_mod_fixed_data.csv',
        'temp_mod_fixed_data_backup.csv',
        'temp_mod_fixed_data_backup_improved.csv',
        'test_recent_temp.csv'
    ]
    
    files_removed = 0
    
    for temp_file in temp_files_to_remove:
        file_path = os.path.join(OUTPUT_DIR, temp_file)
        
        if os.path.exists(file_path):
            try:
                file_size_mb = os.path.getsize(file_path) / (1024*1024)
                os.remove(file_path)
                logger.info(f"   ✅ Eliminado: {temp_file} ({file_size_mb:.2f} MB)")
                files_removed += 1
            except Exception as e:
                logger.warning(f"   ⚠️ No se pudo eliminar {temp_file}: {e}")
        else:
            logger.info(f"   ➖ No existe: {temp_file}")
    
    logger.info(f"\n🎉 Limpieza completada: {files_removed} archivos eliminados")
    
    # Verificar archivo final
    final_file = os.path.join(OUTPUT_DIR, 'data_temp.csv')
    if os.path.exists(final_file):
        file_size_mb = os.path.getsize(final_file) / (1024*1024)
        logger.info(f"✅ Archivo final disponible: data_temp.csv ({file_size_mb:.2f} MB)")
    else:
        logger.error("❌ ADVERTENCIA: No se encuentra data_temp.csv")

# Comentar la siguiente línea si NO quieres ejecutar la limpieza automáticamente
# cleanup_temp_files()

logger.info("✅ Función de limpieza definida (ejecutar manualmente si es necesario)")


2025-06-25 09:27:38 - INFO - ✅ Función de limpieza definida (ejecutar manualmente si es necesario)


In [19]:
# ============================================================================
# 🔧 CORRECCIÓN DEL ARCHIVO data_temp.csv EXISTENTE
# ============================================================================

def fix_existing_data_temp_file():
    """
    Corrige el archivo data_temp.csv existente aplicando la corrección de formato.
    """
    logger.info("🔧 Corrigiendo archivo data_temp.csv existente...")
    logger.info("="*70)
    
    file_path = os.path.join(OUTPUT_DIR, 'data_temp.csv')
    
    if not os.path.exists(file_path):
        logger.error(f"❌ Archivo no encontrado: {file_path}")
        return False
    
    try:
        # Mostrar información del archivo original
        file_size_mb = os.path.getsize(file_path) / (1024*1024)
        logger.info(f"📁 Archivo original: {file_size_mb:.2f} MB")
        
        # Leer archivo
        logger.info("📖 Leyendo archivo...")
        df = pd.read_csv(file_path, index_col='_time', parse_dates=['_time'])
        
        logger.info(f"📊 Registros originales: {len(df):,}")
        logger.info(f"📅 Rango: {df.index.min()} a {df.index.max()}")
        
        # Verificar estado actual (porcentaje de valores válidos por sensor)
        temp_cols = ['1TE416(C)', '1TE417(C)', '1TE418(C)', '1TE419(C)']
        logger.info(f"\\n📊 ESTADO ANTES DE LA CORRECCIÓN:")
        for col in temp_cols:
            if col in df.columns:
                valid_count = df[col].notna().sum()
                percentage = valid_count/len(df)*100
                logger.info(f"🌡️ {col}: {valid_count:,} valores ({percentage:.1f}%)")
        
        # Crear backup del archivo original
        backup_file = file_path.replace('.csv', '_backup_before_fix.csv')
        logger.info(f"\\n💾 Creando backup: {backup_file}")
        df.to_csv(backup_file)
        
        # Aplicar corrección de formato
        logger.info(f"\\n🔧 Aplicando corrección de formato...")
        df_corrected = fix_temperature_format(df)
        
        # Guardar archivo corregido
        logger.info(f"💾 Guardando archivo corregido...")
        df_corrected.to_csv(file_path)
        
        # Mostrar estadísticas finales
        new_file_size_mb = os.path.getsize(file_path) / (1024*1024)
        
        logger.info(f"\\n" + "="*70)
        logger.info(f"✅ CORRECCIÓN COMPLETADA")
        logger.info(f"📊 Registros finales: {len(df_corrected):,}")
        logger.info(f"📁 Tamaño final: {new_file_size_mb:.2f} MB")
        logger.info(f"📅 Rango final: {df_corrected.index.min()} a {df_corrected.index.max()}")
        
        # Estadísticas finales por sensor
        logger.info(f"\\n📊 ESTADO DESPUÉS DE LA CORRECCIÓN:")
        for col in temp_cols:
            if col in df_corrected.columns:
                valid_count = df_corrected[col].notna().sum()
                percentage = valid_count/len(df_corrected)*100
                logger.info(f"🌡️ {col}: {valid_count:,} valores ({percentage:.1f}%)")
        
        logger.info(f"\\n💾 Backup guardado en: {backup_file}")
        logger.info("="*70)
        
        return True
        
    except Exception as e:
        logger.error(f"❌ Error en la corrección: {e}")
        import traceback
        logger.error(f"🔍 Detalles: {traceback.format_exc()}")
        return False

# Ejecutar corrección del archivo existente
logger.info("\\n🔧 INICIANDO CORRECCIÓN DEL ARCHIVO EXISTENTE")
logger.info("="*80)

success = fix_existing_data_temp_file()

if success:
    logger.info("\\n🎉 ¡ARCHIVO CORREGIDO EXITOSAMENTE!")
    logger.info("📋 El archivo data_temp.csv ahora tiene formato consistente")
    logger.info("🧹 Se creó un backup del archivo original por seguridad")
else:
    logger.error("\\n❌ ERROR EN LA CORRECCIÓN")
    logger.error("🔍 Revisa los logs anteriores para más detalles")


2025-06-25 09:27:45 - INFO - \n🔧 INICIANDO CORRECCIÓN DEL ARCHIVO EXISTENTE
2025-06-25 09:27:45 - INFO - ================================================================================
2025-06-25 09:27:45 - INFO - 🔧 Corrigiendo archivo data_temp.csv existente...
2025-06-25 09:27:45 - INFO - ======================================================================
2025-06-25 09:27:45 - INFO - 📁 Archivo original: 27.15 MB
2025-06-25 09:27:45 - INFO - 📖 Leyendo archivo...
2025-06-25 09:27:53 - INFO - 📊 Registros originales: 602,255
2025-06-25 09:27:53 - INFO - 📅 Rango: 2024-07-01 13:00:02+00:00 a 2025-06-25 13:25:00+00:00
2025-06-25 09:27:53 - INFO - \n📊 ESTADO ANTES DE LA CORRECCIÓN:
2025-06-25 09:27:53 - INFO - 🌡️ 1TE416(C): 602,255 valores (100.0%)
2025-06-25 09:27:53 - INFO - 🌡️ 1TE417(C): 602,255 valores (100.0%)
2025-06-25 09:27:53 - INFO - 🌡️ 1TE418(C): 602,255 valores (100.0%)
2025-06-25 09:27:53 - INFO - 🌡️ 1TE419(C): 602,255 valores (100.0%)
2025-06-25 09:27:53 - INFO - \n💾 Creand

In [20]:
# ============================================================================
# 🔢 CORRECCIÓN DE CIFRAS SIGNIFICATIVAS EN DATOS DE TEMPERATURA
# ============================================================================

def normalize_temperature_precision(df, decimal_places=1):
    """
    Normaliza la precisión de los datos de temperatura a un número específico de decimales.
    
    Args:
        df: DataFrame con datos de temperatura
        decimal_places: Número de decimales a mantener (default: 1)
    
    Returns:
        DataFrame con valores redondeados
    """
    logger.info(f"🔢 Normalizando precisión a {decimal_places} decimal(es)...")
    
    try:
        temp_cols = ['1TE416(C)', '1TE417(C)', '1TE418(C)', '1TE419(C)']
        df_normalized = df.copy()
        
        # Mostrar estadísticas antes del redondeo
        logger.info("📊 PRECISIÓN ANTES DEL REDONDEO:")
        for col in temp_cols:
            if col in df_normalized.columns:
                valid_data = df_normalized[col].dropna()
                if not valid_data.empty:
                    min_val = valid_data.min()
                    max_val = valid_data.max()
                    mean_val = valid_data.mean()
                    logger.info(f"🌡️ {col}: Min={min_val:.6f}, Max={max_val:.6f}, Media={mean_val:.6f}")
        
        # Redondear valores de temperatura
        for col in temp_cols:
            if col in df_normalized.columns:
                # Contar valores antes del redondeo
                valid_before = df_normalized[col].notna().sum()
                
                # Aplicar redondeo
                df_normalized[col] = df_normalized[col].round(decimal_places)
                
                # Verificar que no se perdieron valores
                valid_after = df_normalized[col].notna().sum()
                logger.info(f"✅ {col}: {valid_before} → {valid_after} valores válidos")
        
        # Mostrar estadísticas después del redondeo
        logger.info(f"\\n📊 PRECISIÓN DESPUÉS DEL REDONDEO:")
        for col in temp_cols:
            if col in df_normalized.columns:
                valid_data = df_normalized[col].dropna()
                if not valid_data.empty:
                    min_val = valid_data.min()
                    max_val = valid_data.max()
                    mean_val = valid_data.mean()
                    logger.info(f"🌡️ {col}: Min={min_val:.{decimal_places}f}, Max={max_val:.{decimal_places}f}, Media={mean_val:.{decimal_places}f}")
        
        logger.info(f"✅ Precisión normalizada a {decimal_places} decimal(es)")
        return df_normalized
        
    except Exception as e:
        logger.error(f"❌ Error en normalización de precisión: {e}")
        return df

def apply_precision_correction_to_file():
    """
    Aplica corrección de cifras significativas al archivo data_temp.csv existente.
    """
    logger.info("🔢 Aplicando corrección de cifras significativas...")
    logger.info("="*70)
    
    file_path = os.path.join(OUTPUT_DIR, 'data_temp.csv')
    
    if not os.path.exists(file_path):
        logger.error(f"❌ Archivo no encontrado: {file_path}")
        return False
    
    try:
        # Leer archivo
        logger.info("📖 Leyendo archivo...")
        df = pd.read_csv(file_path, index_col='_time', parse_dates=['_time'])
        
        logger.info(f"📊 Procesando {len(df):,} registros...")
        
        # Crear backup antes de la corrección de precisión
        backup_file = file_path.replace('.csv', '_backup_before_precision.csv')
        logger.info(f"💾 Creando backup: {backup_file}")
        df.to_csv(backup_file)
        
        # Aplicar normalización de precisión (1 decimal para temperaturas)
        df_normalized = normalize_temperature_precision(df, decimal_places=1)
        
        # Guardar archivo con precisión normalizada
        logger.info(f"💾 Guardando archivo con precisión corregida...")
        df_normalized.to_csv(file_path)
        
        # Verificar reducción de tamaño del archivo
        original_size = os.path.getsize(backup_file) / (1024*1024)
        new_size = os.path.getsize(file_path) / (1024*1024)
        reduction = ((original_size - new_size) / original_size) * 100
        
        logger.info(f"\\n" + "="*70)
        logger.info(f"✅ CORRECCIÓN DE PRECISIÓN COMPLETADA")
        logger.info(f"📁 Tamaño original: {original_size:.2f} MB")
        logger.info(f"📁 Tamaño nuevo: {new_size:.2f} MB")
        logger.info(f"📉 Reducción: {reduction:.1f}%")
        logger.info(f"💾 Backup guardado en: {backup_file}")
        logger.info("="*70)
        
        return True
        
    except Exception as e:
        logger.error(f"❌ Error en corrección de precisión: {e}")
        import traceback
        logger.error(f"🔍 Detalles: {traceback.format_exc()}")
        return False

# Ejecutar corrección de cifras significativas
logger.info("\\n🔢 INICIANDO CORRECCIÓN DE CIFRAS SIGNIFICATIVAS")
logger.info("="*80)

success = apply_precision_correction_to_file()

if success:
    logger.info("\\n🎉 ¡CIFRAS SIGNIFICATIVAS CORREGIDAS EXITOSAMENTE!")
    logger.info("📋 Los datos de temperatura ahora tienen precisión consistente (1 decimal)")
    logger.info("📉 Tamaño del archivo reducido manteniendo calidad de datos")
    logger.info("💾 Se creó backup del archivo original por seguridad")
else:
    logger.error("\\n❌ ERROR EN LA CORRECCIÓN DE PRECISIÓN")
    logger.error("🔍 Revisa los logs anteriores para más detalles")


2025-06-25 09:30:29 - INFO - \n🔢 INICIANDO CORRECCIÓN DE CIFRAS SIGNIFICATIVAS
2025-06-25 09:30:29 - INFO - ================================================================================
2025-06-25 09:30:29 - INFO - 🔢 Aplicando corrección de cifras significativas...
2025-06-25 09:30:29 - INFO - ======================================================================
2025-06-25 09:30:29 - INFO - 📖 Leyendo archivo...
2025-06-25 09:30:36 - INFO - 📊 Procesando 602,229 registros...
2025-06-25 09:30:36 - INFO - 💾 Creando backup: /home/nicole/SR/SOILING/datos/data_temp_backup_before_precision.csv
2025-06-25 09:30:46 - INFO - 🔢 Normalizando precisión a 1 decimal(es)...
2025-06-25 09:30:46 - INFO - 📊 PRECISIÓN ANTES DEL REDONDEO:
2025-06-25 09:30:46 - INFO - 🌡️ 1TE416(C): Min=0.000000, Max=76.700000, Media=56.268454
2025-06-25 09:30:47 - INFO - 🌡️ 1TE417(C): Min=0.000000, Max=75.900000, Media=53.852447
2025-06-25 09:30:47 - INFO - 🌡️ 1TE418(C): Min=0.000000, Max=76.100000, Media=54.342333
2025-